<a href="https://colab.research.google.com/github/leandroleao2k/ImageML/blob/main/CLIP_exemplo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalação CLIP do OpenAI

In [1]:
!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

/bin/bash: conda: command not found
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 562 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-llwsbj17
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-llwsbj17
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369387 sha256=9294821ab97c3f33a9763a1ec4110f7db1756b851393219554cccdae136d6922
  Stored in directory: /tmp/pip-ephem-wheel-cache-8b7j7r89/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully built clip


In [8]:
import torch
import clip
from PIL import Image

# funcao para mostrar predicoes
def format_preds(probs, texts):
    for p, t in zip(probs, texts):
      printf(f'{p:6.5f}-{t}')

# importando o modelo da internet 300Mb
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

from torchvision import transforms
composed = transforms.Compose([
  transforms.Resize(256),
  transforms.CenterCrop(256)
])

image_pil = composed(Image.open(""))

